In [0]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam


In [0]:
#set to true or false to enable/disable data augmentation
DATA_AUGMENTATION = True

#'cifar', 'fashion mnist', 'mnist'
DATASET_USED = "cifar"

In [19]:
def resize(mnist):
     #function resizes the dimensions of inputted dataset
     data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            data.append(resized_img)
     return data

if DATASET_USED == "cifar":
  # Load CIFAR10 Data
  (X_train_val, y_train_val), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
if DATASET_USED == 'fashion_mnist':
  (X_train_val, y_train_val), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
if DATASET_USED == 'mnist':
  (X_train_val, y_train_val), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

print('train images:', X_train_val.shape) # no.of samples * height * width * channels
print('test images:', X_test.shape) # no.of samples * height * width * channels

# convert labels to onehot-encoding 
y_train_val = tf.keras.utils.to_categorical(y_train_val, num_classes = 10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)
print('train labels:', y_train_val.shape) # no.of samples * num_classes
print('test labels:', y_test.shape) # no.of samples * num_classes

train images: (50000, 32, 32, 3)
test images: (10000, 32, 32, 3)
train labels: (50000, 10)
test labels: (10000, 10)


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.1)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((45000, 32, 32, 3), (5000, 32, 32, 3), (45000, 10), (5000, 10))

In [0]:
def bn_relu_convolution(x, nb_channels, dropout_rate=None, bottleneck=False, weight_decay=1e-4):
    """
    Creates a convolution layers consisting of BN-ReLU-Conv.
    Optional: bottleneck, dropout
    
    """
    # Bottleneck
    if bottleneck:
        bottleneckWidth = 4
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(nb_channels * bottleneckWidth, (1, 1),
                          kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(x)
        # Dropout
        if dropout_rate:
            x = layers.Dropout(dropout_rate)(x)

    # BN-ReLU-Conv
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(nb_channels, (3, 3), padding='same')(x)

    # Dropout
    if dropout_rate:
        x = layers.Dropout(dropout_rate)(x)

    return x

In [0]:
def bn_relu_transition(x, nb_channels, dropout_rate=None, compression=1.0, weight_decay=1e-4):
    """
    Creates a transition layer between dense blocks as transition, which do convolution and pooling.
    Works as downsampling.
    """

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu',)(x)
    x = layers.Convolution2D(int(nb_channels * compression), (1, 1), padding='same',
                             kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(x)

    # Adding dropout
    if dropout_rate:
        x = layers.Dropout(dropout_rate)(x)

    x = layers.AveragePooling2D((2, 2), strides=(2, 2))(x)
    return x

In [0]:
def dense_block(x, num_layers, nb_channels, growth_rate, dropout_rate=None, bottleneck=False,
                    weight_decay=1e-4):
    """
    Creates a dense block and concatenates inputs
    """

    for i in range(num_layers):
        cb = bn_relu_convolution(x, growth_rate, dropout_rate, 
                                 bottleneck) # 1 conv if bottleneck = 0 else 2 conv if bottleneck = 1
        nb_channels += growth_rate
        x = layers.concatenate([cb, x])
    return x, nb_channels

In [0]:
 def DenseNet(input_shape, dense_blocks, dense_layers, growth_rate, compression, bottleneck, 
                     weight_decay, dropout_rate, num_classes, ):
        """
        Build the model
        Returns: tf Keras Model instance
        """

        print('Creating DenseNet with Bottleneck = {}'.format(bottleneck))
        print('#############################################')
        print('No.of. dense blocks: %s' % dense_blocks)
        print('Layers per dense block: %s' % dense_layers)
        print('#############################################')

        # Input Layer
        img_input = layers.Input(shape=input_shape, name = 'img_input')
        nb_channels = growth_rate

        # Input-convolution layer
        x = layers.Conv2D(2 * growth_rate, (3, 3), padding='same', strides=(1, 1),name='input_conv', 
                          kernel_regularizer= tf.keras.regularizers.l2(weight_decay))(img_input)

        # Building dense blocks
        for block in range(dense_blocks - 1):
            # Add dense_block
            x, nb_channels = dense_block(x, dense_layers[block], nb_channels, growth_rate,
                                     dropout_rate, bottleneck, weight_decay) 

            # Add transition
            x = bn_relu_transition(x, nb_channels, dropout_rate, compression, weight_decay) # 1 conv layer
            nb_channels = int(nb_channels * compression)

        # Add last dense block without transition but with only global average pooling
        x, nb_channels = dense_block(x, dense_layers[-1], nb_channels,
                                          growth_rate, dropout_rate, weight_decay)
        
        # prediction of class happens here
        x = layers.BatchNormalization(name = 'prediction_bn')(x)
        x = layers.Activation('relu',  name = 'prediction_relu', )(x)
        x = layers.GlobalAveragePooling2D( name = 'prediction_pool', )(x)
        prediction = layers.Dense(num_classes, name = 'prediction_dense', activation='softmax')(x)

        return tf.keras.Model(inputs=img_input, outputs=prediction, name='densenet')

In [25]:
dense_net = DenseNet(input_shape = (32,32,3), dense_blocks = 3, dense_layers = [16]*3,
                     growth_rate = 12, compression = 0.5, num_classes = 10, bottleneck = True, 
                     dropout_rate = None, weight_decay = 1e-5)
# dense_net.summary()

Creating DenseNet with Bottleneck = True
#############################################
No.of. dense blocks: 3
Layers per dense block: [16, 16, 16]
#############################################


In [0]:
class DenseNet(object):
    
    def __init__(self,input_shape=None, dense_blocks=3, dense_layers=-1, growth_rate=12, num_classes=None,
                 dropout_rate=None, bottleneck=False, compression=1.0, weight_decay=1e-4, depth=40):
        
        # Parameters Check
        if num_classes == None:
            raise Exception(
                'Please define number of classes (e.g. num_classes=10). This is required to create .')

        if compression <= 0.0 or compression > 1.0:
            raise Exception('Compression have to be a value between 0.0 and 1.0.')

        if type(dense_layers) is list:
            if len(dense_layers) != dense_blocks:
                raise AssertionError('Number of dense blocks have to be same length to specified layers')
        elif dense_layers == -1:
            dense_layers = int((depth - 4) / 3)
            if bottleneck:
                dense_layers = int(dense_layers / 2)
            dense_layers = [dense_layers for _ in range(dense_blocks)]
        else:
            dense_layers = [dense_layers for _ in range(dense_blocks)]

        self.dense_blocks = dense_blocks
        self.dense_layers = dense_layers
        self.input_shape = input_shape
        self.growth_rate = growth_rate
        self.weight_decay = weight_decay
        self.dropout_rate = dropout_rate
        self.bottleneck = bottleneck
        self.compression = compression
        self.num_classes = num_classes
        
        
    def build_model(self):
        """
        Build the model
        Returns: tf Keras Model instance
        """
        if self.bottleneck:
            print('Creating DenseNet with Bottlenecks')
        else:
            print('Creating DenseNet without Bottlenecks')
        print('-' * 50)
        print('No.of. dense blocks: %s' % self.dense_blocks)
        print('Layers per dense block: %s' % self.dense_layers)
        print('-'* 50)

        # Input Layer
        img_input = layers.Input(shape = self.input_shape, name = 'img_input')
        nb_channels = self.growth_rate

        # Input-convolution layer
        x = layers.Conv2D(2 * self.growth_rate, (3, 3), padding='same', strides=(1, 1),name='input_conv', 
                          kernel_regularizer= tf.keras.regularizers.l2(self.weight_decay))(img_input)

        # Building dense blocks
        for block in range(self.dense_blocks - 1):
            # Add dense_block
            x, nb_channels = self.dense_block(x, self.dense_layers[block], nb_channels, self.growth_rate,
                                      self.dropout_rate, self.bottleneck, self.weight_decay) 

            # Add transition
            x = self.bn_relu_transition(x, nb_channels, self.dropout_rate, 
                                        self.compression, self.weight_decay) # 1 conv layer
            nb_channels = int(nb_channels * self.compression)

        # Add last dense block without transition but with only global average pooling
        x, nb_channels = self.dense_block(x, self.dense_layers[-1], nb_channels,
                                          self.growth_rate, self.dropout_rate, self.weight_decay)
        
        # prediction of class happens here
        x = layers.BatchNormalization(name = 'prediction_bn')(x)
        x = layers.Activation('relu',  name = 'prediction_relu', )(x)
        x = layers.GlobalAveragePooling2D( name = 'prediction_pool', )(x)
        prediction = layers.Dense(self.num_classes, name = 'prediction_dense', activation='softmax')(x)

        return tf.keras.Model(inputs=img_input, outputs=prediction, name='DenseNet')
        
        
    def dense_block(self, x, num_layers, nb_channels, growth_rate, dropout_rate=None, bottleneck=False,
                    weight_decay=1e-4):
        """
        Creates a dense block and concatenates inputs
        """

        for i in range(num_layers):
            cb = self.bn_relu_convolution(x, growth_rate, dropout_rate, 
                                     bottleneck) # 1 conv if bottleneck = 0 else 2 conv if bottleneck = 1
            nb_channels += growth_rate
            x = layers.concatenate([cb, x])
        return x, nb_channels

        
    def bn_relu_convolution(self, x, nb_channels, dropout_rate=None, bottleneck=False, weight_decay=1e-4):
        """
        Creates a convolution layers consisting of BN-ReLU-Conv.
        Optional: bottleneck, dropout

        """
        # Bottleneck
        if bottleneck:
            bottleneckWidth = 4
            x = layers.BatchNormalization()(x)
            x = layers.Activation('relu')(x)
            x = layers.Conv2D(nb_channels * bottleneckWidth, (1, 1),
                              kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(x)
            # Dropout
            if dropout_rate:
                x = layers.Dropout(dropout_rate)(x)

        # BN-ReLU-Conv
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(nb_channels, (3, 3), padding='same')(x)

        # Dropout
        if dropout_rate:
            x = layers.Dropout(dropout_rate)(x)

        return x

    def bn_relu_transition(self, x, nb_channels, dropout_rate=None, compression=1.0, weight_decay=1e-4):
        """
        Creates a transition layer between dense blocks as transition, which do convolution and pooling.
        Works as downsampling.
        """

        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu',)(x)
        x = layers.Convolution2D(int(nb_channels * compression), (1, 1), padding='same',
                                 kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(x)

        # Adding dropout
        if dropout_rate:
            x = layers.Dropout(dropout_rate)(x)

        x = layers.AveragePooling2D((2, 2), strides=(2, 2))(x)
        return x

In [0]:
dense_net.compile(loss='categorical_crossentropy',
                  optimizer=Adam(),
                  metrics=['accuracy'],)

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, 
                                   horizontal_flip=True, vertical_flip=True)
train_datagen.fit(X_train)
train_data = train_datagen.flow(X_train, y_train, batch_size = 126)

In [0]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_datagen.fit(X_val)
val_data = val_datagen.flow(X_val, y_val, batch_size = 126)

In [0]:
# fits the model on batches with real-time data augmentation:

if DATA_AUGMENTATION == True:
  res_history = dense_net.fit(train_data, epochs = 100,
                              validation_data = (X_val/255.,y_val), steps_per_epoch = 397)
else:
  res_history = dense_net.fit(X_train, y_train, epochs = 100,
                                      validation_data = (X_val,y_val), steps_per_epoch = 397)

Epoch 1/100
397/397 [==============================] - 81s 205ms/step - loss: 1.7790 - accuracy: 0.4672 - val_loss: 2.3908 - val_accuracy: 0.3164
Epoch 2/100
397/397 [==============================] - 79s 198ms/step - loss: 1.3349 - accuracy: 0.6144 - val_loss: 1.4809 - val_accuracy: 0.5538
Epoch 3/100
397/397 [==============================] - 78s 197ms/step - loss: 1.1364 - accuracy: 0.6720 - val_loss: 1.4825 - val_accuracy: 0.5962
Epoch 4/100
397/397 [==============================] - 78s 198ms/step - loss: 1.0154 - accuracy: 0.7077 - val_loss: 1.2718 - val_accuracy: 0.6202
Epoch 5/100
397/397 [==============================] - 78s 197ms/step - loss: 0.9256 - accuracy: 0.7344 - val_loss: 1.2772 - val_accuracy: 0.6432
Epoch 6/100
397/397 [==============================] - 78s 197ms/step - loss: 0.8525 - accuracy: 0.7561 - val_loss: 0.9371 - val_accuracy: 0.7288
Epoch 7/100
397/397 [==============================] - 78s 197ms/step - loss: 0.7999 - accuracy: 0.7746 - val_loss: 0.9085 -

In [0]:
history_dict = res_history.history
print(history_dict)

In [0]:
import matplotlib.pyplot as plt

acc = res_history.history['accuracy']
val_acc = res_history.history['val_accuracy']
loss = res_history.history['loss']
val_loss = res_history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss') # "bo" is for "blue dot"
plt.plot(epochs, val_loss, 'b', label='Validation loss') # b is for "solid blue line"
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [0]:
plt.clf() # clear figure
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [0]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

nb_classes = 10

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.cuda()
        #classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu().detach()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu().detach()])

# Confusion matrix
lbllist= lbllist.numpy()
predlist= predlist.numpy()

print("confusion_matrix:")
conf_mat=confusion_matrix(lbllist, predlist)
print(conf_mat)
print('\n')

# Per-class accuracy
print("per-class accuracy:")
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)
print('\n')
print("prf:")

#PRF
prf = precision_recall_fscore_support(lbllist, predlist)
print("class precision:", prf[0], '\n')
print("class recall:", prf[1], '\n')
print("class f1-score:", prf[2], '\n')